Model simulation Strain_1 (Knock-out speA).By knockint-out speA we delete the conversion of L-arginine into R-arginine, this will increase the concentration of L-arginine in the cell. L-arginine is not involved in the reaction we implemented in the model because it's like a catalyzer of the reaction. However in the real reaction L-arginine is needed by the EFE enzyme, so we want to verify if this knock-out will change the production of Ethylene in our model.

In [24]:
 #import of the model
from cobra.io import read_sbml_model
model = read_sbml_model('modified_model.xml')
model

Name,iJO1366
Memory address,224616b2190
Number of metabolites,1810
Number of reactions,2592
Number of genes,1368
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


Based on the paper the knock-out that will increase L-arginine concentration is argR (repressor of Arginine formation) 

In [25]:
# research of argR gene in our model
argR = None
for gene in model.genes:
    if "ARGR" in gene.name:
        argR = gene.id
        break

print("Gene ID for argR in model:", argR)

Gene ID for argR in model: None


There is no gene for argR, however another gene found in BiGG that will increase L-arginine is speA. In order to knock it out is necessary to know the id of the gene.

In [33]:
# research of speA gene in our model
speA = None
for gene in model.genes:
    if "speA" in gene.name:
        speA = gene.id
        break

print("Gene ID for argR in model:", speA)

Gene ID for argR in model: b2938


In [26]:
#speA gene in the model
model.genes.b2938

Gene identifier,b2938
Name,speA
Memory address,0x224621f8f50
Functional,True
In 1 reaction(s),ARGDCpp


Let's search for the connected reaction to konck it out

In [27]:
#reaction connected to speA gene
model.reactions.ARGDCpp

Reaction identifier,ARGDCpp
Name,Arginine decarboxylase
Memory address,0x224628b1810
Stoichiometry,arg__L_p + h_p --> agm_p + co2_p L-Arginine + H+ --> Agmatine + CO2 CO2
GPR,b2938
Lower bound,0.0
Upper bound,1000.0


Test the production of EFE reaction before the knock-out

In [28]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269851 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.780573951434883 mmol/gDW*h


Knock-out of the reaction connected to speA gene

In [29]:
model.reactions.ARGDCpp.knock_out()
model.reactions.ARGDCpp

Reaction identifier,ARGDCpp
Name,Arginine decarboxylase
Memory address,0x224628b1810
Stoichiometry,arg__L_p + h_p --> agm_p + co2_p L-Arginine + H+ --> Agmatine + CO2 CO2
GPR,b2938
Lower bound,0
Upper bound,0


Maximum theoretical biomass and EFE productivity after the knock-out

In [31]:
with model:
    solution = model.optimize()
    print('Maximum theoretical biomass productivity:', solution.fluxes['BIOMASS_Ec_iJO1366_core_53p95M'], '/h')

    model.objective = model.reactions.EFE_m
    E_production = model.optimize().objective_value
    print('Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction:', E_production, 'mmol/gDW*h')

Maximum theoretical biomass productivity: 0.9823718127269903 /h
Maximum theoretical productivity of Ethylene_forming_enzyme_main reaction: 12.78057395143489 mmol/gDW*h


Maximum theoretical EFE productivity and yield after the knock-out

In [32]:
model.objective = model.reactions.eth_tp
eth_production = model.optimize().objective_value
print('Maximum productivity =', eth_production, 'mmol/gDW*h')
   
eth_maximum_yield = eth_production / (-1*(model.reactions.EX_glc__D_e.flux))
print('Maximum theoretical yield =', eth_maximum_yield, 'mmol-eth/mmol-glc')

Maximum productivity = 12.780573951434917 mmol/gDW*h
Maximum theoretical yield = 1.2780573951434917 mmol-eth/mmol-glc
